這份 Notebook 使用 Ragas 進行 RAG 評估，包括合成評估資料

https://docs.ragas.io/en/latest/index.html

In [ ]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [56]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
!pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━


## 評估資料準備

ragas 除了需要問題集，還需要標準答案。
若要人工製作 dataset，雖然品質好，但是實在太太辛苦了，可以怎麼辦?

可用合成 dataset 策略! 用 LLM 幫我們產生評估資料集

https://docs.ragas.io/en/latest/concepts/testset_generation.html

我們用合成的，原理是:

1. 針對要做 RAG 的文本先拆 chunks，也就是 contexts
2. 針對 context 用 LLM 產生對應的 "問題" 和 "標準答案"

In [4]:
!pip install langchain

In [5]:
!wget https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/113/1130226.pdf

--2024-03-02 10:43:05--  https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/113/1130226.pdf
Resolving www.megabank.com.tw (www.megabank.com.tw)... 104.112.23.93
Connecting to www.megabank.com.tw (www.megabank.com.tw)|104.112.23.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1862439 (1.8M) [application/pdf]
Saving to: ‘1130226.pdf’

1130226.pdf         100%[===================>]   1.78M  --.-KB/s    in 0.03s   

2024-03-02 10:43:05 (51.2 MB/s) - ‘1130226.pdf’ saved [1862439/1862439]



In [6]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.4 MB/s eta 0:00:00


In [7]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("1130226.pdf")
documents = loader.load()

### 合成資料

https://docs.ragas.io/en/stable/getstarted/testset_generation.html

ragas 使用 gpt-4，很燒錢喔

In [8]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/60 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

欄位 ground_truth 就是標準答案

In [9]:
test_df = testset.to_pandas()

In [10]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What are the investment opportunities in the A...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,nan,simple,True
1,How is AI transforming the role of NPCs in the...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,AI is transforming the role of NPCs in the gam...,simple,True
2,What are the concerns and considerations regar...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,The majority of officials are concerned about ...,simple,True
3,What concerns do most officials of the Federal...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,Most officials of the Federal Reserve are conc...,simple,True
4,What is the significance of the acceleration o...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,The significance of the acceleration of corpor...,simple,True
5,"What is AIGC's projected revenue in 2032, as e...",[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,nan,reasoning,True
6,What is the projected growth rate of the gamin...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,The projected growth rate of the gaming indust...,reasoning,True
7,"""What investment opportunities are there in th...",[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,nan,multi_context,True
8,What is the impact of AIGC on the gaming indus...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,AIGC has improved the efficiency of content pr...,multi_context,True
9,"What does ""基金淨值"" represent in investment and h...",[��息日前一日之淨值 ×一年配息次數 ×100%。「年化配息率 」為估算值 。當月配息率計...,nan,reasoning,True


## 準備 Vector DB

In [11]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 

In [16]:
import chromadb
chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="collection3")

In [17]:
def get_embeddings(input, dimensions = 1536, model="text-embedding-3-small"):
  payload = { "input": input, "model": model, "dimensions": dimensions }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else :
    return obj["error"]

In [18]:
import requests
import json
from pprint import pp
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [19]:
loader = PyPDFLoader("1130226.pdf")
text_docs = loader.load()

for page in text_docs:
  chunks = text_splitter.split_text(page.page_content)
  collection.add(
    documents = chunks,
    embeddings = [ get_embeddings(chunk) for chunk in chunks ],
    metadatas = [ { "page": page.metadata['page'], "date": "2023年11月13日" } for x in range( len(chunks) ) ],
    ids=[f"doc-1-page-{page.metadata['page']}-chunk-{x}" for x in range( len(chunks) ) ]
  )

## Baseline

用最基本的相似性搜尋和 gpt-3.5-turbo 來跑跑看

In [20]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [21]:
test_df['used_context'] = None
test_df['used_context'] = test_df['used_context'].astype('object')

for index, row in test_df.iterrows():
    print(f"query {index}")
    results = collection.query(
      query_embeddings = get_embeddings(row["question"]),
      n_results=3
    )
    documents = results['documents'][0]
    context = '\n'.join('* ' + doc for doc in documents)
    prompt= f'''
    Answer the question based on the context below,
    and if the question can't be answered based on the context, say "I don't know"

    Context: {context}

    ---

    Question: {row["question"]}
    Answer (#zh-tw):'''
    messages = [{ "role": "user", "content": prompt } ]
    response = get_completion(messages, model="gpt-3.5-turbo")
    test_df.at[index, 'predict'] = response # 跑出來的預測答案
    test_df.at[index, 'used_context'] = documents # 參考的 contexts

query 0
query 1
query 2
query 3
query 4
query 5
query 6
query 7
query 8
query 9


In [22]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done,used_context,predict
0,What are the investment opportunities in the A...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,nan,simple,True,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,AI產業的投資機會包括投資生產AI晶片的股票以參與AI大趨勢，以及透過定期定額投資把握AI產...
1,How is AI transforming the role of NPCs in the...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,AI is transforming the role of NPCs in the gam...,simple,True,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,AI透過AIGC的發展，有可能轉變成主導遊戲走向的要角，從原本被定位為配角NPC（Non-p...
2,What are the concerns and considerations regar...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,The majority of officials are concerned about ...,simple,True,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,根據聯準會和歐洲央行的會議紀要，他們對於降息過快的擔憂和考量包括：\n1. 官員仍關注通膨未...
3,What concerns do most officials of the Federal...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,Most officials of the Federal Reserve are conc...,simple,True,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,聯準會大多數官員對於降息的擔憂包括通膨未來走勢、讓通膨回到2%目標可能陷入停滯、過早放寬政策...
4,What is the significance of the acceleration o...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,The significance of the acceleration of corpor...,simple,True,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...,2024年企業資本加速投入AI應用的意義在於，這代表企業對於AI技術的重視和投入將會增加，開...
5,"What is AIGC's projected revenue in 2032, as e...",[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,nan,reasoning,True,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...,彭博預估，2032年AIGC的相關營收規模可達1.3兆美元。
6,What is the projected growth rate of the gamin...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,The projected growth rate of the gaming indust...,reasoning,True,[定位為配角 NPC（Non -playable characters ，非玩家角色 ）隨著...,從2022年到2032年，根據Market Research的預測，遊戲業AI產值的年複合成...
7,"""What investment opportunities are there in th...",[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,nan,multi_context,True,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...,根據2024年AI產業預測的增長和企業加大資本投資，投資機會包括投資生產AI晶片的股票以參與...
8,What is the impact of AIGC on the gaming indus...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...,AIGC has improved the efficiency of content pr...,multi_context,True,[定位為配角 NPC（Non -playable characters ，非玩家角色 ）隨著...,AIGC提升了產製內容的效率，觸角甚至延伸到遊戲設計。早期AI在遊戲產業中往往被定位為配角N...
9,"What does ""基金淨值"" represent in investment and h...",[��息日前一日之淨值 ×一年配息次數 ×100%。「年化配息率 」為估算值 。當月配息率計...,nan,reasoning,True,[付本金 、利息或破產而蒙受虧損 。部分基金不適合無法承擔相關風險之投資人 。基金經理公司以...,基金淨值代表基金資產價值減去負債後的淨值。它可能因市場因素而上下波動，投資人因不同時間進場，...


## 轉成 eval 用的 dataset 格式，準備跑評估

In [23]:
eval_df = test_df[['question', 'ground_truth', 'predict', 'used_context']]

In [24]:
eval_df

,question,ground_truth,predict,used_context
0,What are the investment opportunities in the A...,nan,AI產業的投資機會包括投資生產AI晶片的股票以參與AI大趨勢，以及透過定期定額投資把握AI產...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
1,How is AI transforming the role of NPCs in the...,AI is transforming the role of NPCs in the gam...,AI透過AIGC的發展，有可能轉變成主導遊戲走向的要角，從原本被定位為配角NPC（Non-p...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
2,What are the concerns and considerations regar...,The majority of officials are concerned about ...,根據聯準會和歐洲央行的會議紀要，他們對於降息過快的擔憂和考量包括：\n1. 官員仍關注通膨未...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
3,What concerns do most officials of the Federal...,Most officials of the Federal Reserve are conc...,聯準會大多數官員對於降息的擔憂包括通膨未來走勢、讓通膨回到2%目標可能陷入停滯、過早放寬政策...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
4,What is the significance of the acceleration o...,The significance of the acceleration of corpor...,2024年企業資本加速投入AI應用的意義在於，這代表企業對於AI技術的重視和投入將會增加，開...,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...
5,"What is AIGC's projected revenue in 2032, as e...",nan,彭博預估，2032年AIGC的相關營收規模可達1.3兆美元。,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...
6,What is the projected growth rate of the gamin...,The projected growth rate of the gaming indust...,從2022年到2032年，根據Market Research的預測，遊戲業AI產值的年複合成...,[定位為配角 NPC（Non -playable characters ，非玩家角色 ）隨著...
7,"""What investment opportunities are there in th...",nan,根據2024年AI產業預測的增長和企業加大資本投資，投資機會包括投資生產AI晶片的股票以參與...,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...
8,What is the impact of AIGC on the gaming indus...,AIGC has improved the efficiency of content pr...,AIGC提升了產製內容的效率，觸角甚至延伸到遊戲設計。早期AI在遊戲產業中往往被定位為配角N...,[定位為配角 NPC（Non -playable characters ，非玩家角色 ）隨著...
9,"What does ""基金淨值"" represent in investment and h...",nan,基金淨值代表基金資產價值減去負債後的淨值。它可能因市場因素而上下波動，投資人因不同時間進場，...,[付本金 、利息或破產而蒙受虧損 。部分基金不適合無法承擔相關風險之投資人 。基金經理公司以...


In [25]:
# 改欄位名，配合 eval 方法
eval_df = eval_df.rename(columns={'used_context': 'contexts'})
eval_df = eval_df.rename(columns={'predict': 'answer'})

In [26]:
eval_df

,question,ground_truth,answer,contexts
0,What are the investment opportunities in the A...,nan,AI產業的投資機會包括投資生產AI晶片的股票以參與AI大趨勢，以及透過定期定額投資把握AI產...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
1,How is AI transforming the role of NPCs in the...,AI is transforming the role of NPCs in the gam...,AI透過AIGC的發展，有可能轉變成主導遊戲走向的要角，從原本被定位為配角NPC（Non-p...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
2,What are the concerns and considerations regar...,The majority of officials are concerned about ...,根據聯準會和歐洲央行的會議紀要，他們對於降息過快的擔憂和考量包括：\n1. 官員仍關注通膨未...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
3,What concerns do most officials of the Federal...,Most officials of the Federal Reserve are conc...,聯準會大多數官員對於降息的擔憂包括通膨未來走勢、讓通膨回到2%目標可能陷入停滯、過早放寬政策...,[本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決...
4,What is the significance of the acceleration o...,The significance of the acceleration of corpor...,2024年企業資本加速投入AI應用的意義在於，這代表企業對於AI技術的重視和投入將會增加，開...,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...
5,"What is AIGC's projected revenue in 2032, as e...",nan,彭博預估，2032年AIGC的相關營收規模可達1.3兆美元。,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...
6,What is the projected growth rate of the gamin...,The projected growth rate of the gaming indust...,從2022年到2032年，根據Market Research的預測，遊戲業AI產值的年複合成...,[定位為配角 NPC（Non -playable characters ，非玩家角色 ）隨著...
7,"""What investment opportunities are there in th...",nan,根據2024年AI產業預測的增長和企業加大資本投資，投資機會包括投資生產AI晶片的股票以參與...,[預估，生成式 AI佔全球企業的支出比重將在 2032年由目前不到 3%提升至 12%，20...
8,What is the impact of AIGC on the gaming indus...,AIGC has improved the efficiency of content pr...,AIGC提升了產製內容的效率，觸角甚至延伸到遊戲設計。早期AI在遊戲產業中往往被定位為配角N...,[定位為配角 NPC（Non -playable characters ，非玩家角色 ）隨著...
9,"What does ""基金淨值"" represent in investment and h...",nan,基金淨值代表基金資產價值減去負債後的淨值。它可能因市場因素而上下波動，投資人因不同時間進場，...,[付本金 、利息或破產而蒙受虧損 。部分基金不適合無法承擔相關風險之投資人 。基金經理公司以...


In [27]:
from datasets import Dataset

eval_dataset = Dataset.from_pandas(eval_df)

## 開始跑評估

In [30]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

## 分項分數

In [31]:
result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [32]:
result

{'context_precision': 0.5000, 'faithfulness': 0.9167, 'answer_relevancy': 0.4617, 'context_recall': 0.6000}

### End-to-End 分數

In [35]:
from ragas.metrics import AnswerSimilarity
answer_similarity = AnswerSimilarity()

In [ ]:
answer_similarity.score(eval_dataset)

In [43]:
from ragas.metrics import AnswerCorrectness
answer_correctness = AnswerCorrectness(
    weights=[0.4,0.6]
)

In [ ]:
answer_correctness.score(eval_dataset)